## Linear Conjugate Gradient Method (CGM)

The linear conjugate gradient method is used for solving a linear system of equations $$Ax = b$$
where $A$ is an $n\times n$ symmetric positive definite matrix.  

Such an $x$ can be determined by solving the equivalent convex minimization problem $$ \min_{x\in\mathbb{R}^n} \phi(x) = \frac{1}{2} x^T A x - b^T x.$$  


Thus, we can interpret the iterative Linear Conjugate Gradient algorithm as a technique for solving linear systems or as a means for minimizing convex quadratic functions.  

Note, that $\nabla \phi$ is the residual of the linear system, i.e. $$\nabla \phi (x) = Ax - b = r(x),$$

and for a given $x_k$ we say $r_k = r(x_k)$.

The CGM generates a _conjugacy_ set of nonzero vectors $\{p_0, p_1,...,p_l\}$ in an economical fashion, a very remarkable property of the method.
Having a set of vectors conjugate to $A$ will allow us to minimize $\phi$ in $n$ steps by successively minimizing it along the individual directions in the conjugate set.  

***def:*** A set of nonzeros vectors $\{p_0, p_1,...,p_l\}$ is said to be _conjugate_ to a SPD matrix $A$ if $p^T_iAp_j = 0 ~ \forall i\not = j$.  


Let $x_o \in \mathbb{R}^n$ be a given starting point, and let $\{p_0, p_1,...,p_l\}$ be a set of conjugate directions with respect to $A$.
Then we define the sequence $(x_k)$ by 

$$x_{k+1} = x_k + \alpha_k p_k,$$  
where $\alpha_k$ is one-dimensional minimizer of $\phi$ given by  
$$\alpha_k = -\frac{r_k^T p_k}{p_k^TAp_k}.$$  

&nbsp;  
***Theorem 5.1:*** For any $x_0 \in \mathbb{R}^n$ the sequence $(x_k)$ converges to the global minimizer solution $x^*$ in at most n steps.  
+ The recursively generated sequence $(x_k)$ is reffered to as the **conjugate direction algorithm**.  

&nbsp;  
***Theorem 5.2:*** For any starting point $x_0 \in \mathbb{R}^n$ and suppose $(x_k)$ is generated using the conjugate direction algorithm. Then  

$$r_k^T p_i = 0 ~ \forall i \in [0, k-1],$$  

and $x_k$ is the minimizer of $\phi(x) = \frac{1}{2} x^T A x - b^T x$ over the set $\{x ~|~ x = x_o + \text{span}\{p_0, p_1,..., p_{k-1}\} \}$  
+ This asserts that the current residual $r_k$ is orthogonal to all previous search directions, a crucial property to the algorithms utility.


&nbsp;  

If the Hessian of $\phi$ happens to be a diagonal (note, $\nabla^2 \phi = A$), then the set of basis vectors will suffice for a conjugate direction set.
When this property doesn't hold, the eigenvectors are both conjugate and orthogonal to the span of $A$ but this is generally an expensive computation.
We could drop the orthogonality requirement and modify the Gram-Schmidt algorithm to produce a conjugate set, but that has trade-off's in terms of space complexity.  


A beter approach would be to use the **CGM**, which is both a conjugate direction method and a procedure for generating a conjugate set.
The CGM only requires knowledge of the previous conjugate vector, namely, given p_{k-1} it can produce a new vector $p_k$ such that $p_{j}^T A p_k = 0$ for $j < k$.
This is done, by picking the next conjugate vector by a linear combination of the residual $r_k$ (which is $\nabla \phi(x_k)$ and the previous conjugate direction, i.e.  

$$p_k = - r_k + \beta_k p_{k-1},$$

where the scalar $\beta_k$ is to be determined by the requirement that $p_{k-1}$ and $p_k$ must be conjugate with respect to A.
By premultiplying $p_{k-1}^T A$ to both sides of the previous equation we can solve for $\beta_k$ as  

$$\beta_k = \frac{r_k^T A p_{k-1}}{p_{k-1}^T A p_{k-1}}.$$

Lastly, we can safely choose the initial direction $p_o$ by the fact that $\phi$ is convex, namely, $p_0 = \nabla \phi(x_0)$ where $x_0$ is the initial guess.

We now can specify a preliminary conjugate gradient method! But always remeber that it is not the gradients that are conjugate, rather, the search directions that are conjugate!

**function cgm_linear($A$, $b$, $x$, $\epsilon$):** Parameters are $A \in \mathbb{R}^{n \times n}$ SPD, $x, b \in \mathbb{R^n}$, and $\epsilon$ is a specified precision. The algorithm terminates when it has identified an global minimum or more than $n$ iterations have occured. It returns $x^*$, the determined global minimizer. 

&nbsp;  







In [125]:
using LinearAlgebra

function cgm_linear(A, b, x, eps) 
    # initialize variables to the kth iteration
    r = A*x-b
    p = -r
    k  = 0
    while norm(r) > eps
        x -= ((r'*p) / (p'*A*p)) * p
        r = A*x - b
        p = -r + ((r'*A*p) / (p'*A*p)) * p  
        if (k+=1) > length(x) # theroem 2.1 asserts the LHS should be near length(x), i.e. n?
            println("cgm_linear: terminating before meeting specified precision")
            break
        end
    end
    println("Number of iterations k = ", k)
    x
end

cgm_linear (generic function with 1 method)

In [131]:
# perform some basic testing of cgm_linear

n = 20
A1 = Diagonal(rand(n))
b1 = rand(n)
x1 = ones(n)

x_star = cgm_linear(A1, b1, x1, 10^(-3))
println("Euclidean error: ", norm(A1\b1 - x_star))


Number of iterations k = 11
Euclidean error: 0.004677132925527356


In [139]:
# perform another test with a dense matrix.. this one regularly terminates early on large system?
# but we are close, for small n around 20!

temp = rand(n,n)
A2 = (temp + temp')
b2 = rand(n)
x2 = ones(n)

x_star = cgm_linear(A2, b2, x2, 10^(-3))
println("Euclidean error: ", norm(A2\b2 - x_star))

cgm_linear: terminating before meeting specified precision
Number of iterations k = 21
Euclidean error: 7.527414390706144e-8


Using the fact that the residuals at each iteration are mutually orthogonal, and since each search direction $p_k$ and residual $r_k$ is contained in the _Krylov subspace of degree k for $r_0$_ ***defined*** as  

$$ \kappa (r_0; k) \equiv \text{span } \{r_0, Ar_0,...., A^k r_0\}, $$

we can devise a more efficient linear CGM variant by the next theorem.

&nbsp;  
***Theorem 5.3:***  Suppose that the $k$th iterate generated by the conjugate gradient method is not the solution point $x^*$. The following properties hold:  

$$
\begin{aligned}
r_k^Tr_i &= 0, \text{ for } i = 0, 1, ..., k-1, \\
\text{span } \{r_0, r_1,..., r_l \} & = \text{span }\{r_0, Ar_0,...., A^k r_0\}, \\
\text{span } \{p_0, p_1,..., p_l \} & = \text{span }\{r_0, Ar_0,...., A^k r_0\}, \\
p^t_k A p_i & = 0, \text{ for } i = 0, 1, ..., k-11. \\
\end{aligned}
$$

Therefore, the sequence $(x_k)$ converges to $x^*$ in at most n steps.  

The proof is by induction, relying on the fact the the first direction $p_0$ is the steepest descent directions $-r_0 = -\nabla \phi$. Furthermore, it is a necessitiy that our $p_0$ is chosen this way, otherwise the result doesn't hold.  
&nbsp;
Using our new results, we can use a new step:

$$ \alpha_k = \frac{r^T_kr_k}{p^T_k A p_k}. $$

We can also simply our formula for $\beta_{k+1}$:

$$ \beta_{k+1} = \frac{r^T_{k+1} r_{k+1}}{r^T_k r_k}. $$


We make the update, and devise the ***function cgm_lin_fast***

In [137]:
function cgm_linear_fast(A, b, x, eps) 
    # initialize variables to the kth iteration
    r = A*x-b
    p = -r
    k  = 0
    while norm(r) > eps
        a = (r'*r) / (p'*A*p)
        x +=  a* p
        r_old = r
        r = r + a*A*p
        p = -r + ((r'*r) / (r_old'r_old)) * p  
        if (k+=1) > length(x) # theroem 2.1 asserts the LHS should be near length(x), i.e. n?
            println("cgm_linear: terminating before meeting specified precision")
            break
        end
    end
    println("Number of iterations k = ", k)
    x
end

cgm_linear_fast (generic function with 1 method)

In [140]:
x_star = cgm_linear_fast(A1, b1, x1, 10^(-3))
println("Euclidean error: ", norm(A1\b1 - x_star), "\n")


x_star = cgm_linear_fast(A2, b2, x2, 10^(-3))
println("Euclidean error: ", norm(A2\b2 - x_star))

Number of iterations k = 11
Euclidean error: 0.004677132925527564

cgm_linear: terminating before meeting specified precision
Number of iterations k = 21
Euclidean error: 3.0236157920638187e-6


The results appear to be quite identical between the two variants!